In [1]:
import tempfile
import subprocess
import os, sys
import numpy as np
import pickle
import pprint

In [2]:
import tensorflow as tf

/home/xinyug2/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xinyug2/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xinyug2/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xinyug2/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [42]:
# env_var = os.environ
# print("User's Environment variable:")
# pprint.pprint(dict(env_var), width = 1)

'/mnt/c/Users/xinyu/netsurfp-1.0d.Linux/netsurfp-1.0/bin'

In [113]:
POLARITY       = {'A': -0.06, 'R': -0.84, 'N': -0.48, 'D': -0.80, 'C': 1.36, 'Q': -0.73,
                  'E': -0.77, 'G': -0.41, 'H': 0.49, 'I': 1.31, 'L': 1.21, 'K': -1.18, 
                  'M': 1.27, 'F': 1.27, 'P': 0.0, 'S': -0.50, 'T': -0.27, 'W': 0.88, 
                  'Y': 0.33, 'V': 1.0}
HYDROPHOBICITY = {'A': -0.40, 'R': -0.59, 'N': -0.92, 'D': -1.31, 'C': 0.17, 'Q': -0.91,
                  'E': -1.22, 'G': -0.67, 'H': -0.64, 'I': 1.25, 'L': 1.22, 'K': -0.67, 
                  'M': 1.02, 'F': 1.92, 'P': -0.49, 'S': -0.55, 'T': -0.28, 'W': 0.50, 
                  'Y': 1.67, 'V': 0.91}
VOLUME         = {'A': 52.6, 'R': 109.1, 'N': 75.7, 'D': 68.4, 'C': 68.3, 'Q': 89.7,
                  'E': 84.7, 'G': 36.3, 'H': 91.9, 'I': 102.0, 'L': 102.0, 'K': 105.1, 
                  'M': 97.7, 'F': 113.9, 'P': 73.6, 'S': 54.9, 'T': 71.2, 'W': 135.4, 
                  'Y': 116.2, 'V': 85.1, 'X': 52.6, 'Z': 52.6, 'B': 52.6, 'J': 102.0}
BLOSUM50       = {'A': [ 5, -2, -1, -2, -1, -1, -1,  0, -2, -1, -2, -1, -1, -3, -1,  1,  0, -3, -2,  0],
                  'R': [-2,  7, -1, -2, -4,  1,  0, -3,  0, -4, -3,  3, -2, -3, -3, -1, -1, -3, -1, -3],
                  'N': [-1, -1,  7,  2, -2,  0,  0,  0,  1, -3, -4,  0, -2, -4, -2,  1,  0, -4, -2, -3],
                  'D': [-2, -2,  2,  8, -4,  0,  2, -1, -1, -4, -4, -1, -4, -5, -1,  0, -1, -5, -3, -4],
                  'C': [-1, -4, -2, -4, 13, -3, -3, -3, -3, -2, -2, -3, -2, -2, -4, -1, -1, -5, -3, -1],
                  'Q': [-1,  1,  0,  0, -3,  7,  2, -2,  1, -3, -2,  2,  0, -4, -1,  0, -1, -1, -1, -3],
                  'E': [-1,  0,  0,  2, -3,  2,  6, -3,  0, -4, -3,  1, -2, -3, -1, -1, -1, -3, -2, -3],
                  'G': [ 0, -3,  0, -1, -3, -2, -3,  8, -2, -4, -4, -2, -3, -4, -2,  0, -2, -3, -3, -4],
                  'H': [-2,  0,  1, -1, -3,  1,  0, -2, 10, -4, -3,  0, -1, -1, -2, -1, -2, -3,  2, -4],
                  'I': [-1, -4, -3, -4, -2, -3, -4, -4, -4,  5,  2, -3,  2,  0, -3, -3, -1, -3, -1,  4],
                  'L': [-2, -3, -4, -4, -2, -2, -3, -4, -3,  2,  5, -3,  3,  1, -4, -3, -1, -2, -1,  1],
                  'K': [-1,  3,  0, -1, -3,  2,  1, -2,  0, -3, -3,  6, -2, -4, -1,  0, -1, -3, -2, -3],
                  'M': [-1, -2, -2, -4, -2,  0, -2, -3, -1,  2,  3, -2,  7,  0, -3, -2, -1, -1,  0,  1],
                  'F': [-3, -3, -4, -5, -2, -4, -3, -4, -1,  0,  1, -4,  0,  8, -4, -3, -2,  1,  4, -1],
                  'P': [-1, -3, -2, -1, -4, -1, -1, -2, -2, -3, -4, -1, -3, -4, 10, -1, -1, -4, -3, -3],
                  'S': [ 1, -1,  1,  0, -1,  0, -1,  0, -1, -3, -3,  0, -2, -3, -1,  5,  2, -4, -2, -2],
                  'T': [ 0, -1,  0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1,  2,  5, -3, -2,  0],
                  'W': [-3, -3, -4, -5, -5, -1, -3, -3, -3, -3, -2, -3, -1,  1, -4, -4, -3, 15,  2, -3],
                  'Y': [-2, -1, -2, -3, -3, -1, -2, -3,  2, -1, -1, -2,  0,  4, -3, -2, -2,  2,  8, -1],
                  'V': [ 0, -3, -3, -4, -1, -3, -3, -4, -4,  4,  1, -3,  1, -1, -3, -2,  0, -3, -1,  5],
                  '-': [-5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5]}
amino_acids    = ('G', 'A', 'V', 'L', 'I', 'M', 'F', 'W', 'P', 'S',
                  'T', 'C', 'Y', 'N', 'Q', 'D', 'E', 'K', 'R', 'H',)

SSnetsurfp     = ('H', 'E', 'C',)
# NETSURFP_BIN  = os.environ.get('NETSURFP_BIN',  'netsurfp')
NETSURFP_BIN  = '/mnt/c/Users/xinyu/netsurfp-1.0/netsurfp'
RF_MODEL      = None

In [114]:
NETSURFP_BIN

'/mnt/c/Users/xinyu/netsurfp-1.0/netsurfp'

In [115]:
def run_cmd(cmd, input_string=''):
    """Run the cmd with input_string as stdin and return output."""
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stdin=subprocess.PIPE,
                         stderr=subprocess.PIPE, universal_newlines=True, close_fds=True)
    out, stderr = p.communicate(input=input_string)
    if p.returncode:
        raise Exception('Cmd {} failed: {}'.format(cmd[0], stderr))
    return out

In [116]:
def encode(char,code,sparse):
    """ Encoding characters

    Usage::
        >>> char, code, sparse = 'A', 2, 'molecular'
        >>> encode(char, code, sparse)
        [-0.06, -0.4, 52.6]
    """

    if code == 0:
        return char
    elif code == 1:
        SS = SSnetsurfp
        return encode_sparse(char, SS)
    elif code == 2:
        if sparse == 'sparse':
            return encode_sparse(char, amino_acids)
        elif sparse == 'blosum':
            return BLOSUM50[char]
        else:
            return seq_to_codes(char)
    elif code == 3:
        return encode_sparse(char, antibody_chain)
    elif code == 4:
        return encode_sparse(char, CDRpos)
    elif code == 5:
        return [CDRindex.index(char)]

In [117]:
def encode_sparse(char, types):
    """Sparse encode a character"""
    
    codings = {AA: [1 if i == j else 0 for j in range(len(types))]
        for i, AA in enumerate(types)}
    gap = [0] * len(types)

    return codings.get(char.upper(), gap)

In [118]:
def seq_to_codes(char):
    """Converts sequence to a list of polarity, hydrophobicity, volume"""
    code = list()

    try:
        code = [POLARITY[char], HYDROPHOBICITY[char], VOLUME[char]]
    except KeyError:
        code = [0, 0, 0]

    return code

In [119]:
def rolling_mean(pred):
    """Performs the rolling window to smooth antigen predictions

    Usage::
        >>> predictions = [0.1,0.1,0.5,0.6,0.5]                            
        >>> rolling_mean(predictions)
        [0.20000000000000004, 0.20000000000000004, 0.19999999999999998, 0.20000000000000001, 0.19999999999999998]
    """
    
    _pred = list()

    for i, p in enumerate(pred):
        window = [p]

        for j in range(4,0,-1):
            if i-j < 0:
                window.append(0)
            else:
                window.append(pred[i-j])
        for j in range(1,5):
            try:
                window.append(pred[i+j])
            except IndexError:
                window.append(0)

        _pred.append(np.mean(np.array(window)))

    return _pred


In [120]:
def get_neighbours(position, chain, surface, windowsize):
    """Returns the neighbours of a given residue in antigen sequence"""
    
    right_window = {'res': list(), 'rsa': list(), 'ss': list()}
    left_window  = {'res': list(), 'rsa': list(), 'ss': list()}
    window       = list()

    for i in range(int((windowsize-1)/2),0,-1):
        #Left side
        if position-i < 0:
            left_window['res'].extend('-')
            left_window['rsa'].append(1.0)
            left_window['ss'].extend('X')
        else:
            left_window['res'].extend(chain[position-i])
            left_window['rsa'].append(surface[(position-i)+1][0][1])
            left_window['ss'].extend(surface[(position-i)+1][1][0])

        #Right side
        try:
            right_window['res'].extend(chain[position+i])
            right_window['rsa'].append(surface[(position+i)+1][0][1])
            right_window['ss'].extend(surface[(position+i)+1][1][0])
        except IndexError:
            right_window['res'].extend('-')
            right_window['rsa'].append(1.0)
            right_window['ss'].extend('X')
    rsa_window = left_window['rsa'] + [surface[position+1][0][1]] + list(reversed(right_window['rsa']))
    ss_window  = left_window['ss'] + [surface[position+1][1][0]] + list(reversed(right_window['ss']))
    res_window = left_window['res'] + [chain[position]] + list(reversed(right_window['res']))
  
    encoded = list()
    for res in res_window:
        encoded.extend(seq_to_codes(res))
    encoded.extend(rsa_window)
    for ss in ss_window:
        encoded.extend(encode_sparse(ss, SSnetsurfp))


    return encoded

In [121]:
def get_volume(chain):
    """Calculates the sequential volume from sequence"""

    return sum([ VOLUME[res.upper()] for res in chain ])

def get_surface(chain):
    """Predicts the surface accessibility from sequence"""
    surface = dict()
    tempfsa = tempfile.NamedTemporaryFile()
    tmp_string = '>temp\n{}'.format(chain).encode()
#     print(tmp_string)
    tempfsa.write(tmp_string)
    tempfsa.flush()
#     print(tempfsa.name)
    out = run_cmd([NETSURFP_BIN, '-i', tempfsa.name, '-a'])
    
#     out = run_cmd([NETSURFP_BIN])
    print(out)
    out = out.split('\n')
    for line in out:
        line = line.strip() 
        if line == '' or line[0] == '#':
            continue
        line = line.split()
        ss = ''
        index = 7
        if len(line) < 10:
            ss = 'X'
            rsa = 'X'
            index = 6
            sspreds = [0.0, 0.0, 0.0]
        elif float(line[index]) > float(line[index+1]) and float(line[index]) > float(line[index+2]):
            ss = 'H'
            rsa = line[0]
            sspreds = [line[index], line[index+1], line[index+2]]
        elif float(line[index+1]) > float(line[index]) and float(line[index+1]) > float(line[index+2]):
            ss = 'E'
            rsa = line[0]
            sspreds = [line[index], line[index+1], line[index+2]]
        elif float(line[index+2]) > float(line[index+1]) and float(line[index+2]) > float(line[index]):
            ss = 'C'
            rsa = line[0]
            sspreds = [line[index], line[index+1], line[index+2]]

        surface[int(line[index-4])] = [[rsa, float(line[index-3])], [ss]+sspreds]

    return surface

In [122]:
class Antigen(object):
    """
    Class object of a single antigen sequence.
    """

    def __init__(self, entry_name, sequence):
        self.entry_name  = entry_name
        self.sequence    = sequence.upper()
        self.description = ""
        self.features    = []
        self.netsurfp    = {}
        self.predicted   = []
        
    def pred_netsurfp(self):
        """Initialises a dict of list containing results from NetsurfP {POSITION: [RSA, SS],..}"""
        try:
            self.netsurfp = get_surface(self.sequence)
        except:
            err = 'NetsurfP ({}) failed to predict: {}\n{}'
            raise BPFeatureError(err.format(NETSURFP_BIN,self.entry_name, self.sequence))

    def get_features(self):
        """Setups the features to predict sequence."""

        agvolume = get_volume(self.sequence)
        dataset  = list()
        for agpos in range(len(self.sequence)):
            window = get_neighbours(agpos, self.sequence, self.netsurfp, 9)
            dataset.append(window+[agvolume])

        self.features = dataset

    def predict(self):
        """"""
        try:
            preds = bp.utils.RF_MODEL.predict(self.features)
        except:
            err = 'RandomForest failed to predict: {}\n{}'
            raise bp.BPPredictError(err.format(self.entry_name, self.sequence))

        self.predicted = rolling_mean(preds)

In [128]:
name = '1'
antigen_sequence = 'cdafvgtwKLVssenfddymkevgvgfatrkvagMAKpnmiisvngdlvtirsesTfknteisfklgvefdeitaddrkvksiitldggalvqvqkwdgksttikrkrdgdklvvecvmkgvtstrvyErA'
antigen = Antigen(name, antigen_sequence)

In [129]:
antigen.pred_netsurfp()

In [97]:
antigen.sequence

'CDAFVGTWKLVSSENFDDYMKEVGVGFATRKVAGMAKPNMIISVNGDLVTIRSESTFKNTEISFKLGVEFDEITADDRKVKSIITLDGGALVQVQKWDGKSTTIKRKRDGDKLVVECVMKGVTSTRVYERA'

In [130]:
antigen.netsurfp

{}